# Group Project
#### **FINN43815 - Python for Finance**

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import kurtosis, skew
import yfinance as yf

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Breakout Strategy (with Moving Average)

#### 1. Import Data

In [2]:
data = pd.read_csv('cleaned_data.csv')

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_2890/1299969751.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('cleaned_data.csv')


In [3]:
# drop unnecessary columns
data = data[["date", "permno", "price"]]
data

,date,permno,price
0,1990-01-31,0111145D UN Equity,20.6875
1,1990-02-28,0111145D UN Equity,22.0000
2,1990-03-30,0111145D UN Equity,19.5000
3,1990-04-30,0111145D UN Equity,17.6250
4,1990-05-31,0111145D UN Equity,19.8125
...,...,...,...
514691,2022-04-29,J UN Equity,138.5500
514692,2022-05-31,J UN Equity,140.0900
514693,2022-06-30,J UN Equity,127.1300
514694,2022-07-29,J UN Equity,137.3000


#### 2. Calculate monthly returns

In [4]:
data['rets'] = data.groupby('permno')['price'].transform(pd.Series.pct_change)
data.head()

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_2890/1728035520.py:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  data['rets'] = data.groupby('permno')['price'].transform(pd.Series.pct_change)


,date,permno,price,rets
0,1990-01-31,0111145D UN Equity,20.6875,NaN
1,1990-02-28,0111145D UN Equity,22.0000,0.063444
2,1990-03-30,0111145D UN Equity,19.5000,-0.113636
3,1990-04-30,0111145D UN Equity,17.6250,-0.096154
4,1990-05-31,0111145D UN Equity,19.8125,0.124113


#### 3. Calculate moving averages

In [5]:
data["sm5"] = data.groupby("permno")["price"].rolling(window=5, min_periods=5).mean().reset_index(drop=True)
data["sm15"] = data.groupby("permno")["price"].rolling(window=15, min_periods=15).mean().reset_index(drop=True)
data

,date,permno,price,rets,sm5,sm15
0,1990-01-31,0111145D UN Equity,20.6875,NaN,NaN,NaN
1,1990-02-28,0111145D UN Equity,22.0000,0.063444,NaN,NaN
2,1990-03-30,0111145D UN Equity,19.5000,-0.113636,NaN,NaN
3,1990-04-30,0111145D UN Equity,17.6250,-0.096154,NaN,NaN
4,1990-05-31,0111145D UN Equity,19.8125,0.124113,19.925,NaN
...,...,...,...,...,...,...
514691,2022-04-29,J UN Equity,138.5500,0.005370,200.662,192.782667
514692,2022-05-31,J UN Equity,140.0900,0.011115,186.042,193.828667
514693,2022-06-30,J UN Equity,127.1300,-0.092512,180.462,194.789333
514694,2022-07-29,J UN Equity,137.3000,0.079997,178.242,195.424000


In [6]:
data = data.dropna()

#### 4. Identify Trading Signals

In [7]:
data["signal_buy"] = np.where((data.price > data.sm5) & (data.price > data.sm15), 1, 0) # buy
data["signal_sell"] = np.where((data.price < data.sm5) & (data.price < data.sm15), -1, 0) # sell
data

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_2890/2486862408.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["signal_buy"] = np.where((data.price > data.sm5) & (data.price > data.sm15), 1, 0) # buy
/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_2890/2486862408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["signal_sell"] = np.where((data.price < data.sm5) & (data.price < data.sm15), -1, 0) # sell


,date,permno,price,rets,sm5,sm15,signal_buy,signal_sell
14,1991-03-28,0111145D UN Equity,20.5625,-0.003030,21.3875,20.675000,0,-1
15,1991-04-30,0111145D UN Equity,21.3750,0.039514,21.0375,20.720833,1,0
16,1991-05-31,0111145D UN Equity,21.0000,-0.017544,20.8375,20.654167,1,0
17,1991-06-28,0111145D UN Equity,21.6250,0.029762,21.0375,20.795833,1,0
18,1991-07-31,0111145D UN Equity,21.0625,-0.026012,21.1250,21.025000,0,0
...,...,...,...,...,...,...,...,...
514691,2022-04-29,J UN Equity,138.5500,0.005370,200.6620,192.782667,0,-1
514692,2022-05-31,J UN Equity,140.0900,0.011115,186.0420,193.828667,0,-1
514693,2022-06-30,J UN Equity,127.1300,-0.092512,180.4620,194.789333,0,-1
514694,2022-07-29,J UN Equity,137.3000,0.079997,178.2420,195.424000,0,-1


In [8]:
data.loc[:,"position_buy"] = data.groupby("permno")["signal_buy"].diff()
data.loc[:,"position_sell"] = data.groupby("permno")["signal_sell"].diff()
# position = 1 -> buy
# position = -1 -> sell

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_2890/3520999582.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,"position_buy"] = data.groupby("permno")["signal_buy"].diff()
/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_2890/3520999582.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,"position_sell"] = data.groupby("permno")["signal_sell"].diff()


In [9]:
data.position_buy = data.position_buy.replace(-1, 0)
data.position_sell = data.position_sell.replace(1, 0)

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_2890/108836894.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.position_buy = data.position_buy.replace(-1, 0)
/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_2890/108836894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.position_sell = data.position_sell.replace(1, 0)


In [10]:
data = data.dropna()

In [11]:
data["position"] = np.where(data.position_buy == 1, 1, 0)
data["position"] = np.where(data.position_sell == -1, -1, data.position)
data.position

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_2890/576510106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["position"] = np.where(data.position_buy == 1, 1, 0)
/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_2890/576510106.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["position"] = np.where(data.position_sell == -1, -1, data.position)


15        1
16        0
17        0
18        0
19       -1
         ..
514691    0
514692    0
514693    0
514694    0
514695    0
Name: position, Length: 174356, dtype: int64

In [12]:
# shift position (trading decision) by 1 month.
data.loc[:,"position"] = data.groupby("permno")["position"].shift(1)

In [13]:
data

,date,permno,price,rets,sm5,sm15,signal_buy,signal_sell,position_buy,position_sell,position
15,1991-04-30,0111145D UN Equity,21.3750,0.039514,21.0375,20.720833,1,0,1.0,0.0,NaN
16,1991-05-31,0111145D UN Equity,21.0000,-0.017544,20.8375,20.654167,1,0,0.0,0.0,1.0
17,1991-06-28,0111145D UN Equity,21.6250,0.029762,21.0375,20.795833,1,0,0.0,0.0,0.0
18,1991-07-31,0111145D UN Equity,21.0625,-0.026012,21.1250,21.025000,0,0,0.0,0.0,0.0
19,1991-08-30,0111145D UN Equity,20.8125,-0.011869,21.1750,21.091667,0,-1,0.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
514691,2022-04-29,J UN Equity,138.5500,0.005370,200.6620,192.782667,0,-1,0.0,0.0,0.0
514692,2022-05-31,J UN Equity,140.0900,0.011115,186.0420,193.828667,0,-1,0.0,0.0,0.0
514693,2022-06-30,J UN Equity,127.1300,-0.092512,180.4620,194.789333,0,-1,0.0,0.0,0.0
514694,2022-07-29,J UN Equity,137.3000,0.079997,178.2420,195.424000,0,-1,0.0,0.0,0.0


In [14]:
holding = []
for _,df in data[["permno", "position"]].groupby("permno"):
    p = np.zeros(df.shape[0])
    for i in range(1,df.shape[0]):
        if (data.position.iloc[i] != -1 and p[i-1] == 1) or data.position.iloc[i] == 1:
            p[i] = 1
        else:
            p[i] = 0
    holding = holding + list(p)

In [15]:
data["holding"] = holding

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_2890/3805329903.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["holding"] = holding


In [16]:
# get returns for strategy
data["br_rets"] = data["holding"] * data.rets

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_2890/2616624591.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["br_rets"] = data["holding"] * data.rets


#### 5. Calculate Trading Return

In [17]:
strategy_returns = data.groupby("date")["br_rets"].mean().reset_index()

In [18]:
strategy_returns.date = pd.to_datetime(strategy_returns.date)

In [19]:
strategy_returns = strategy_returns.dropna()

In [20]:
strategy_returns["br_cum_rets"] = (strategy_returns.br_rets +1).cumprod()

### Evaluation

In [21]:
px.line(strategy_returns, y='br_cum_rets', x='date', labels={'br_cum_rets': 'Cumulative returns'}, title = 'Breakout Returns').show()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [22]:
px.histogram(strategy_returns[['br_rets']]).show()

In [23]:
monthly_rets = strategy_returns[["date", "br_rets"]].resample('M', on='date').mean() # Calculate average daily return for each month
monthly_rets['Colour']='red' # Add column with 'red' value
monthly_rets.loc[monthly_rets['br_rets']>=0, 'Colour']='green' # Replace 'red' with 'green' where return is non-negative
px.bar(monthly_rets,x=monthly_rets.index,y="br_rets").update_traces(marker_color=monthly_rets["Colour"]).show()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [24]:
# mean and volatility
mean_ret = strategy_returns['br_rets'].mean()
volatility = strategy_returns['br_rets'].std()
print(mean_ret, volatility)

0.008662566509835252 0.03474952243226871


In [25]:
print('Annualised expected return is: '+str(round(100*mean_ret*12,2)) +'%')
print('Annualised volatility is: '+str(round(100*volatility*np.sqrt(12),2)) +'%')

Annualised expected return is: 10.4%
Annualised volatility is: 12.04%


In [26]:
# describing statistics of returns
(strategy_returns['br_rets']*100).describe()

count    377.000000
mean       0.866257
std        3.474952
min      -14.428098
25%       -1.007375
50%        1.145625
75%        2.867692
max       14.488596
Name: br_rets, dtype: float64

In [27]:
# skewness
print(skew(strategy_returns['br_rets'].dropna()))

-0.45957876469402287


In [28]:
# kurtosis
print(kurtosis(strategy_returns["br_rets"].dropna()))

2.690930073028099


In [29]:
# Value at Risk
VaR = strategy_returns['br_rets'].quantile(0.005)
VaR

-0.1172203014984387

In [30]:
for q in [0.1, 0.05, 0.01, 0.001, 0.0001]:
    VaR = strategy_returns['br_rets'].quantile(q)
    print('There is a '+str(round(100*q,3))+'% chance that my losses will be '+str(round(VaR*100, 2))+'% or worse over the next day.')

There is a 10.0% chance that my losses will be -3.11% or worse over the next day.
There is a 5.0% chance that my losses will be -4.89% or worse over the next day.
There is a 1.0% chance that my losses will be -8.79% or worse over the next day.
There is a 0.1% chance that my losses will be -14.24% or worse over the next day.
There is a 0.01% chance that my losses will be -14.41% or worse over the next day.


In [31]:
VaR = strategy_returns['br_rets'].quantile(0.005)
rets_tail = strategy_returns.loc[strategy_returns['br_rets']<=VaR,["date", "br_rets", "br_cum_rets"]]
rets_tail

,date,br_rets,br_cum_rets
210,2008-10-31,-0.144281,5.051109
347,2020-03-31,-0.139368,12.469063


In [32]:
ES = rets_tail['br_rets'].mean()
ES

-0.14182424912141245

In [33]:
for q in [0.1, 0.05, 0.01, 0.001, 0.0001]:
    VaR = strategy_returns['br_rets'].quantile(q)
    rets_tail = strategy_returns.loc[strategy_returns['br_rets']<=VaR,:]
    ES = rets_tail['br_rets'].mean()
    print('I expect to lose on average '+ str(round(ES*100, 2))+'% over 1-day period given that I have exceeded my VaR for '+str(round(100*q,3))+'%.')

I expect to lose on average -5.86% over 1-day period given that I have exceeded my VaR for 10.0%.
I expect to lose on average -7.82% over 1-day period given that I have exceeded my VaR for 5.0%.
I expect to lose on average -12.21% over 1-day period given that I have exceeded my VaR for 1.0%.
I expect to lose on average -14.43% over 1-day period given that I have exceeded my VaR for 0.1%.
I expect to lose on average -14.43% over 1-day period given that I have exceeded my VaR for 0.01%.


In [34]:
# according to bloomberg the risk free rate (treasury yields) 4.76 % (last 30 years)
# calculation of sharpe ratio

rf = 0.0476
sharpe_ratio = (mean_ret-rf)/volatility
sharpe_ratio

-1.1205170823875008

In [35]:
# get SP500 data from 1990-01-31 to 2022-08-30	
sp500_ticker = yf.Ticker("^GSPC")
sp500 = sp500_ticker.history(start="1990-02-28", end="2022-09-30", interval="1mo")
sp500 = sp500[["Close"]]
sp500["Rets"] = sp500.Close.pct_change()

In [36]:
sp500

,Close,Rets
Date,,
1990-03-01 00:00:00-05:00,339.940002,NaN
1990-04-01 00:00:00-05:00,330.799988,-0.026887
1990-05-01 00:00:00-04:00,361.230011,0.091989
1990-06-01 00:00:00-04:00,358.019989,-0.008886
1990-07-01 00:00:00-04:00,356.149994,-0.005223
...,...,...
2022-05-01 00:00:00-04:00,4132.149902,0.000053
2022-06-01 00:00:00-04:00,3785.379883,-0.083920
2022-07-01 00:00:00-04:00,4130.290039,0.091116


In [37]:
# Calculate the average return of the market
average_market_return = sp500.Rets.mean()

# Calculate the covariance between portfolio and market returns
covariance = strategy_returns['br_rets'].cov(sp500['Rets'].reset_index(drop=True))

# Calculate the variance of the market returns
market_variance = sp500['Rets'].var()

# Calculate the portfolio's beta
portfolio_beta = covariance / market_variance

# Calculate Jensen's Alpha
jensens_alpha = mean_ret - (rf + portfolio_beta * (average_market_return - rf))

print("Portfolio Beta:", portfolio_beta)
print("Jensen's Alpha:", jensens_alpha)

Portfolio Beta: -0.04520592647117319
Jensen's Alpha: -0.04077337434621996


In [38]:
# treynor ratio
treynor_ratio = (mean_ret - rf)/portfolio_beta
treynor_ratio

0.8613347082930438

In [39]:
# maximum drawdown
peak = strategy_returns.reset_index(drop=True).loc[1, "br_cum_rets"]
max_drawdown = 0
for ret in strategy_returns["br_cum_rets"].dropna():
    if ret >= peak:
        peak = ret
    else:
        drawdown = (peak - ret)/peak
        max_drawdown = max(max_drawdown, drawdown)

print(max_drawdown)

0.4023273823507834


In [40]:
# calmar-ratio
annualised_exp_return = round(mean_ret*12,2)
calmar_ratio = (annualised_exp_return - rf)/max_drawdown
calmar_ratio

0.13024219155511818